In [ ]:
import numpy as np
import pandas as pd
import math
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
from GwoOptimizer import GWO

In [ ]:
# Load the data
db = pd.read_csv('london_merged.csv')

db['timestamp'] = pd.to_datetime(db['timestamp'], format ="%Y-%m-%d %H:%M:%S")
db['day'] = pd.to_numeric(db['timestamp'].dt.strftime('%d'))
db['month'] = db['timestamp'].dt.month
db['hour'] = db['timestamp'].dt.hour
db['day_of_week'] = db['timestamp'].dt.dayofweek
print(db.head())

db['cnt_log'] = np.log1p(db['cnt'])
sns.distplot(db['cnt_log'])
plt.title("Histogram of logarithm of count of shared bikes")
_ = plt.ylabel('Density')

db.plot(x='timestamp',y='cnt')
plt.title('Count of bikes by date')
plt.ylabel('Shares bikes')
plt.show()

date_start = '2015-07-09 00:00:00'
date_end = '2015-07-09 23:00:00'
db = db[(db['timestamp'] < date_start) | (db['timestamp'] > date_end)]

date_start = '2015-08-06 00:00:00'
date_end = '2015-08-06 23:00:00'
db = db[(db['timestamp'] < date_start) | (db['timestamp'] > date_end)]
db.plot(x='timestamp',y='cnt')
plt.title('Count of bikes by date')
plt.ylabel('Shares bikes')
plt.show()

db['weather_code'] = pd.Categorical(db['weather_code'])
db['season'] = pd.Categorical(db['season'])

weather_code = pd.get_dummies(db['weather_code'], prefix = 'weather')
season = pd.get_dummies(db['season'], prefix = 'season')

db = pd.concat([db, weather_code], axis=1)
db = pd.concat([db, season], axis=1)

db = db.drop(['t2','is_weekend','cnt','timestamp','weather_code','season'], axis = 1)
db.head()

x = db.drop(['cnt_log'], axis=1)
y = db['cnt_log']

scaler = preprocessing.MinMaxScaler()
x = pd.DataFrame(scaler.fit_transform(x), columns = x.columns)


In [ ]:
#The index corresponding to the training percent of the data is obtained
train_percent = 0.7
split_index = int(train_percent * len(db))
#Data (x) and targets (y) for training and test phases are obtained
train_x = x.iloc[:split_index].values
test_x = x.iloc[split_index:].values

train_y = y.iloc[:split_index].values
test_y = y.iloc[split_index:].values

print ('train_x shape:', train_x.shape)
print ('train_y shape:', train_y.shape)
print ('test_x shape:', test_x.shape)
print ('test_y shape:', test_y.shape)


In [ ]:
# Feature selection using GWO
fit = GWO(fitness_function, 0, 1, num_features, 10, 20)
selected_features = np.where(fit>0.4)[0]
print('features',selected_features)